# Лабораторная работа №2
## Выполнил студент группы БФИ2001 Заморский Пётр Владимирович

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
3. [Задание 3](#Задание-№3)

### Задание №1
Реализовать методы поиска в соответствии с заданием. Организовать генерацию начального набора случайных данных. Для всех вариантов добавить реализацию добавления, поиска и удаления элементов. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [1]:
import random
import time
arr = [random.randrange(0, 10000) for _ in range(10000)]
sorted_arr = arr.copy()
sorted_arr.sort()
test_repeat = 10000

def test_unsorted(find_func):
    test(find_func, arr)
def test_sorted(find_func):
    test(find_func, sorted_arr)

def test(find_func, arr):
    start_time = time.time()
    
    for i in range(test_repeat):
        target_index = random.randrange(0, len(arr))
        target_value = sorted_arr[target_index]
        finded_index = find_func(arr, target_value)
        result = arr[finded_index] == target_value
        if not result:
            print(f"Value finded wrong, target index: '{target_index}', finded: '{finded_index}'")
    
    total_time = time.time() - start_time
    print(f"len {len(arr)} x repeats {test_repeat}\n{round(total_time*1000)} ms")
    
def profile_time(func, title):
    start_time = time.time()
    func()
    total_time = time.time() - start_time
    print(f"{title}: {round(total_time*1000)} ms")

In [2]:
#Бинарный поиск
def binary_find(arr, value):
    if value < arr[0] or value > arr[len(arr) - 1]:
        return -1
    return binary_find_iteration(arr, value, 0, len(arr) - 1)

def binary_find_iteration(arr, value, start, end):
    if end - start < 0: return -1
    mid = start + (end - start) // 2
    mid_value = arr[mid]
    if value == mid_value:
        return mid
    if value < mid_value:
        return binary_find_iteration(arr, value, start, mid - 1)
    
    return binary_find_iteration(arr, value, mid + 1, end)

def binary_add(arr, value):
    def iteration(arr, value, start, end):
        mid = start + (end - start) // 2
        mid_value = arr[mid]
        if end - start <= 0:
            if mid_value < value: return mid + 1
            return mid
        if value == mid_value: return mid
        if value < mid_value:
            return iteration(arr, value, start, mid - 1)
        return iteration(arr, value, mid + 1, end)
    
    i = iteration(arr, value, 0, len(arr) - 1)
    arr.insert(i, value)
    
def binary_remove(arr, value):
    i = binary_find(arr, value)
    if i >= 0: 
        arr.pop(i)
        return True
    return False


test_sorted(binary_find)

len 10000 x repeats 10000
61 ms


In [3]:
#Бинарное дерево
class BinarySearchTree:
    def __init__(self, array = []):
        self.root = None
        for i in range(len(array)):
            self.insert(i, array[i])
    
    def insert(self, index, value):
        if self.root is None:
            self.root = BinaryTreeNode(index, value, None)
            return
        self.root.insert(index, value)
        
    def find(self, value):
        node = self.find_node(value)
        if node is None: return -1
        return node.index
    
    def find_node(self, value):
        if self.root is None: return None
        return self.root.find_node(value)
    
    def remove(self, value):
        node = self.find_node(value)
        if node is None: return False
        self.remove_node(node)
        return True
    
    def remove_node(self, node):
        if node.left is None:
            self.subtree_shift(node, node.right)
        elif node.right is None:
            self.subtree_shift(node, node.left)
        else:
            successor = node.successor()
            if successor.parent != node:
                self.subtree_shift(successor, successor.right)
                successor.right = node.right
                successor.right.parent = successor
            self.subtree_shift(node, successor)
            successor.left = node.left
            successor.left.parent = successor
    
    def subtree_shift(self, u, v):
        if u.parent is None:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        if v is not None:
            v.parent = u.parent
        
class BinaryTreeNode:
    def __init__(self, index, value, parent):
        self.left = None
        self.right = None
        self.index = index
        self.value = value
        self.parent = parent
    
    def insert(self, index, value):
        if value > self.value:
            if self.right is None:
                self.right = BinaryTreeNode(index, value, self)
                return
            self.right.insert(index, value)
        else:
            if self.left is None:
                self.left = BinaryTreeNode(index, value, self)
                return
            self.left.insert(index, value)
            
    def find_node(self, value):
        if value == self.value:
            return self
        
        if value > self.value:
            if self.right is None: return None
            return self.right.find_node(value)
        
        if self.left is None: return None
        return self.left.find_node(value)
    
    def successor(self):
        if self.right is not None:
            return self.minimum()
        x = self
        y = self.parent
        while y is not None and x == y.right:
            x = y
            y = parent
        return y
    
    def minimum(self):
        if self.left is None: return self
        return self.left.minimum()
        

tree = None

def init(): 
    global tree
    tree = BinarySearchTree(arr)
        
profile_time(init, "Init")
test_unsorted(lambda arr, value: tree.find(value))

Init: 42 ms
len 10000 x repeats 10000
56 ms


In [4]:
#Метод Фибоначчи
def initial_fibonacci():
    return 1, 1, 2

def next_fibonacci(f):
    return f[-2], f[-1], f[-2] + f[-1]

def prev_fibonacci(f):
    return f[-2] - f[-3], f[-3], f[-2]

def fibonacci_find(arr, value):
    left = 0
    right = len(arr) - 1
    last = arr[right]
    f = initial_fibonacci()
    
    while len(arr) > f[-1] - 1:
        f = next_fibonacci(f)
    
    max_f = f[-1] - 1
    
    while left <= right:
        mid = left + f[-2] - 1
        if mid >= len(arr):
            mid_value = last
        else:
            mid_value = arr[mid]
        if mid_value > value:   
            right = mid - 1
            f = prev_fibonacci(f)
        elif mid_value < value:
            left = mid + 1
            f = prev_fibonacci(prev_fibonacci(f))
        else:
            if mid >= len(arr) - 1: 
                return len(arr) - 1
            return mid
    return -1

def fibonacci_add(arr, value):
    left = 0
    right = len(arr) - 1
    last = arr[right]
    f = initial_fibonacci()
    
    while len(arr) > f[-1] - 1:
        f = next_fibonacci(f)
    
    max_f = f[-1] - 1
    i = left
    
    while left <= right:
        mid = left + f[-2] - 1
        if mid >= len(arr):
            mid_value = last
        else:
            mid_value = arr[mid]
        if mid_value > value:   
            right = mid - 1
            f = prev_fibonacci(f)
            i = mid
        elif mid_value < value:
            left = mid + 1
            i = mid + 1
            f = prev_fibonacci(prev_fibonacci(f))
        else:
            if mid >= len(arr) - 1: 
                i = len(arr) - 1
            else:
                i = mid
            break
    
    arr.insert(i, value)
    
def fibonacci_remove(arr, value):
    i = fibonacci_find(arr, value)
    if i >= 0: 
        arr.pop(i)
        return True
    return False
            
test_sorted(fibonacci_find)

len 10000 x repeats 10000
163 ms


In [5]:
#Интерполяционный
def interpolate_find(arr, value):
    return interpolate_find_iteration(arr, value, 0, len(arr) - 1)

def interpolate_find_iteration(arr, value, start, end):
    if not (arr[start] <= value <= arr[end]): return -1
    mid = start + (value - arr[start]) * (end - start) // (arr[end] - arr[start])
    mid_value = arr[mid]
    if value == mid_value:
        return mid
    if value < mid_value:
        return interpolate_find_iteration(arr, value, start, mid - 1)
    
    return interpolate_find_iteration(arr, value, mid + 1, end)

def interpolate_add(arr, value):
    def iteration(arr, value, start, end):
        if arr[start] >= value: 
            print(1)
            return start
        if value >= arr[end]: 
            print(2)
            return end
        mid = start + (value - arr[start]) * (end - start) // (arr[end] - arr[start])
        mid_value = arr[mid]
        if value == mid_value:
            print(3)
            return mid
        if value < mid_value:
            return iteration(arr, value, start, mid - 1)

        return iteration(arr, value, mid + 1, end)
    
    i = iteration(arr, value, 0, len(arr) - 1)
    arr.insert(i, value)
    
def interpolate_remove(arr, value):
    i = interpolate_find(arr, value)
    if i >= 0: 
        arr.pop(i)
        return True
    return False

test_sorted(interpolate_find)

len 10000 x repeats 10000
36 ms


In [6]:
# Стандартная

test_unsorted(lambda arr, value: arr.index(value))

len 10000 x repeats 10000
503 ms


### Задание №2
Написать соответствующие решения для заданных методов.

In [7]:
#Простое рехэширование

class SimpleHashTable:
    def __init__(self):
        self.size = 128
        self.count = 0
        self.arr = [None for _ in range(self.size)]
        
    def add(self, key, value):
        size = self.size
        if self.count / size > 0.8:
            size *= 2
            self.set_size(size)
        h = hash(key) % size
        e = self.arr[h]
        while e is not None:
            if e[0] == key: return False
            h += 1
            h %= size
            e = self.arr[h]
        self.arr[h] = (key, value)
        self.count += 1
        return True
    
    def remove(self, key):
        size = self.size
        h = hash(key) % size
        e = self.arr[h]
        c = 0
        while (e is None or e[0] != key) and c < self.size:
            h += 1
            h %= size
            c += 1
            e = self.arr[h]
        if c == self.size: return False
        self.arr[h] = None
        self.count -= 1
        return True
    
    def get(self, key):
        size = self.size
        h = hash(key) % size
        e = self.arr[h]
        c = 0
        while (e is None or e[0] != key) and c < self.size:
            h += 1
            h %= size
            c += 1
            e = self.arr[h]   
        if c == self.size: return None
        return e[1]
    
    def set_size(self, new_size):
        old_arr = self.arr
        self.size = new_size
        self.count = 0
        self.arr = [None for _ in range(self.size)]
        for e in old_arr:
            if e is None: continue
            self.add(e[0], e[1])
    
def hash_str(key):
    result = 0
    for i in range(len(key)):
        result ^= ord(key[i])
    return result


table = SimpleHashTable()



start_time = time.time()

for i in range(10000):
    table.add(str(i), i)

total_time = time.time() - start_time
print(f"Insert: {round(total_time*1000)} ms")

start_time = time.time()

for i in range(10000):
    table.get(str(i))

total_time = time.time() - start_time
print(f"Get: {round(total_time*1000)} ms")

start_time = time.time()

for i in range(10000):
    table.remove(str(i))

total_time = time.time() - start_time
print(f"Remove: {round(total_time*1000)} ms")

Insert: 33 ms
Get: 11 ms
Remove: 12 ms


In [8]:
#Рехэширование с помощью псевдослучайных чисел

class RandomHashTable:
    def __init__(self):
        self.size = 128
        self.count = 0
        self.arr = [None for _ in range(self.size)]
        
    def add(self, key, value):
        size = self.size
        if self.count / size > 0.8:
            size *= 2
            self.set_size(size)
        h = hash(key) % size
        r = 1
        e = self.arr[h]
        while e is not None:
            if e[0] == key: return False
            r *= 5
            r %= size * 4
            e = self.arr[(h + (r >> 2)) % size]
        self.arr[(h + (r >> 2)) % size] = (key, value)
        self.count += 1
        return True
    
    def remove(self, key):
        size = self.size
        h = hash(key) % size
        r = 1
        e = self.arr[h]
        c = 0
        while (e is None or e[0] != key) and c < self.size:
            c += 1
            r *= 5
            r %= size * 4
            e = self.arr[(h + (r >> 2)) % size]
        if c == self.size: return False
        self.arr[(h + (r >> 2)) % size] = None
        self.count -= 1
        return True
    
    def get(self, key):
        size = self.size
        h = hash(key) % size
        r = 1
        e = self.arr[h]
        c = 0
        while (e is None or e[0] != key) and c < self.size:
            c += 1
            r *= 5
            r %= size * 4
            e = self.arr[(h + (r >> 2)) % size] 
        if c == self.size: return None
        return e[1]
    
    def set_size(self, new_size):
        old_arr = self.arr
        self.size = new_size
        self.count = 0
        self.arr = [None for _ in range(self.size)]
        for e in old_arr:
            if e is None: continue
            self.add(e[0], e[1])


table = RandomHashTable()    

start_time = time.time()

for i in range(10000):
    table.add(str(i), i)

total_time = time.time() - start_time
print(f"Insert: {round(total_time*1000)} ms")

start_time = time.time()

for i in range(10000):
    table.get(str(i))

total_time = time.time() - start_time
print(f"Get: {round(total_time*1000)} ms")

start_time = time.time()

for i in range(10000):
    table.remove(str(i))

total_time = time.time() - start_time
print(f"Remove: {round(total_time*1000)} ms")

Insert: 37 ms
Get: 13 ms
Remove: 14 ms


In [9]:
#Метод цепочек

class ChainHashTable:
    def __init__(self):
        self.size = 128
        self.count = 0
        self.arr = [None for _ in range(self.size)]
        
    def add(self, key, value):
        size = self.size
        if self.count / size > 0.8:
            size *= 2
            self.set_size(size)
        h = hash_str(key) % size
        e = self.arr[h]
        if e is None:
            self.arr[h] = [key, value, None]
            self.count += 1
            return True
        while e[2] is not None:
            e = e[2]
        e[2] = [key, value, None]
        self.count += 1
    
    def remove(self, key):
        size = self.size
        h = hash_str(key) % size
        prev_element = None
        e = self.arr[h]
        while e is not None:
            if e[0] == key:
                if prev_element is not None:
                    prev_element[2] = e[2]
                return True
            prev_element = e
            e = e[2]
        return False
    
    def get(self, key):
        size = self.size
        h = hash_str(key) % size
        e = self.arr[h]
        while e is not None:
            if e[0] == key: return e[1]
            e = e[2]
        return None
    
    def set_size(self, new_size):
        old_arr = self.arr
        self.size = new_size
        self.count = 0
        self.arr = [(None) for _ in range(self.size)]
        for i in range(len(old_arr)):
            e = old_arr[i]
            while e is not None:
                self.add(e[0], e[1])
                e = e[2]


table = ChainHashTable()

start_time = time.time()

for i in range(10000):
    table.add(str(i), i)

total_time = time.time() - start_time
print(f"Insert: {round(total_time*1000)} ms")

start_time = time.time()

for i in range(10000):
    table.get(str(i))

total_time = time.time() - start_time
print(f"Get: {round(total_time*1000)} ms")

start_time = time.time()

for i in range(10000):
    table.remove(str(i))

total_time = time.time() - start_time
print(f"Remove: {round(total_time*1000)} ms")

Insert: 286 ms
Get: 271 ms
Remove: 18 ms


### Задание №3
Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям
Написать программу,  которая находит хотя бы один способ решения задач.


In [10]:
def queens():
    n = 8
    y = 0
    a = []
    l = [0 for _ in range(8)]
    repeats = 0
    
    def is_correct(x, y):
        for q in a:
            if q[0] == x or abs(q[0] - x) == abs(q[1] - y):
                return False
        return True
    
    def check_row(): 
        nonlocal y
        nonlocal n
        for x in range(l[y], 8):
            if is_correct(x, y):
                l[y] = x + 1
                a.append([x, y])
                n -= 1
                y += 1
                return True
        return False
    
    while n > 0:
        r = check_row()
        if r == False:
            a.pop()
            n += 1
            y -= 1
            for i in range(y + 1, len(l)):
                l[i] = 0
                
        repeats += 1
    print("Repeats: ", repeats)
    
    return a

print(queens())

Repeats:  218
[[0, 0], [4, 1], [7, 2], [5, 3], [2, 4], [6, 5], [1, 6], [3, 7]]
